In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'
%load_ext tensorboard.notebook

In [ ]:
learning_rate = 0.001
epochs = 50
slope = 3
logs_path = '/home/akshit/Documents/summaries/example1'

A little cleanup for tensorboard. If the log directory exists, remove it and create a new one

In [ ]:
import shutil, os
if os.path.exists(logs_path):
    shutil.rmtree(logs_path)
os.mkdir(logs_path)

Let's generate our own input data, to make sure the points don't actually end up on a prefect line themselves, let's also add some 'noise' to it.

In [ ]:
n_samples = 50
train_x = np.linspace(1, 20, n_samples)
train_y = slope*train_x + 4*np.random.randn(n_samples)

np.linspace(start, stop , num) returns num evenly spaced numbers from start to stop. num is optional. Default is 50.
The slope is 3 (random choice, so the equation is y = 3x) and to distort it, we added random guassian distributed numbers. 

randn generates an array of shape (d0, d1, ..., dn), filled with random floats sampled from a univariate “normal” (Gaussian) distribution of mean 0 and variance 1 (if any of the d_i are floats, they are first converted to integers by truncation). A single float randomly sampled from the distribution is returned if no argument is provided.

In our case randn returns n_samples no of random nos.

In [ ]:
plt.plot(train_x, train_y, 'o')
plt.plot(train_x, slope*train_x) #the original line equation
plt.show()

Define weights(slope) and biases(y intercept, in this case it's 0)

Define the input data placeholders, the weight and bias variables.

In [ ]:
with tf.name_scope("X"):
    X = tf.placeholder(tf.float32)
with tf.name_scope("Y"):
    Y = tf.placeholder(tf.float32)
    
with tf.name_scope("Weights"):
    W = tf.Variable(np.random.randn(), name='weights')
with tf.name_scope("Biases"):
    B = tf.Variable(np.random.randn(), name='bias')

Now let's define the graph

In [ ]:
#pred = X*W + B #is the basic idea
pred = tf.add(tf.multiply(X, W), B)

Now comes the cost function, let's minimize the sum

In [ ]:
with tf.name_scope("Cost_function"):
    cost = tf.reduce_sum((pred - Y)**2)/(2*n_samples)

That's 
\begin{equation*}
\frac{1}{2n}\sum_{i}^n \left(pred\left(i\right)-Y_i \right)^2
\end{equation*}

or choose any cost function you like

Now let's choose an optimiser

In [ ]:
with tf.name_scope("Optimizer"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    #or use AdamOptimizer

Wel'll run this in a loop. But first, let's intilize all our varibales.

In [ ]:
init = tf.global_variables_initializer()

Define our summaries and an op to merge them all

In [ ]:
tf.summary.scalar('Cost_plot',cost)
tf.summary.histogram('Weight_hist',W)
tf.summary.histogram('Bias_hist',B)
merge_op = tf.summary.merge_all()

Now let's define our regression

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    file_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    c = 0.0
    cost_plot = []
    x_plot = range(0,epochs+1)
    
    weight_plot = []
    bias_plot = []
    
    for i in range(epochs+1):
        for x,y in zip(train_x, train_y):
            _, c, w, b, summary = sess.run([optimizer, cost, W, B, merge_op], feed_dict={X: x, Y: y})
            
        cost_plot.append(c)
        weight_plot.append(w)
        bias_plot.append(b)
        if not i % 5:
            file_writer.add_summary(summary, i)
            print "epoch:{:3d} cost={:.4f} w={:.4f} b={:.4f}" .format(i,c,w,b)
    
    print "\n....Done...."

Plot the regression that our model learnt.

In [ ]:
plt.plot(train_x, train_y, 'o')
plt.plot(train_x, w*train_x + b, label='Regression')
plt.plot(train_x, slope*train_x, label='Original', linestyle = 'dashed')
plt.legend()
plt.show()

Launch TensorBoard

In [ ]:
%tensorboard --logdir {logs_path}

Let's take a look at the cost plot for each epoch

In [ ]:
plt.plot(x_plot, cost_plot)
plt.xlabel('Epochs')
plt.ylabel('Cost')
plt.show()

The original line had a slope of 3 (weight) and y intercept (the bias) of 0.5
Let's see how well our model did after each epoch.

In [ ]:
plt.plot(x_plot, weight_plot, label='Weight')
plt.plot(x_plot, bias_plot, label='Bias')
plt.axhline(y=slope, color='black', linestyle='dashed')
#plt.axhline(y=0.5, color='black', linestyle='dashed')
plt.xlabel('Epochs >>>')
plt.legend()
plt.ylim(top=5)
plt.show()